In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from processclass import Experiment2D
from pso_optimizer_class import PSO_Optimizer
from analyse import deposit_fwhm, get_peak

Generating reference data

In [ ]:
pr = Experiment2D()
# Initializing model
pr.n0 = 2.7  # 1/nm^2
pr.F = 730.0  # 1/nm^2/s
pr.s = 1.0
pr.V = 0.05  # nm^3
pr.tau = 200e-6  # s
pr.D = 12e5  # nm^2/s
pr.sigma = 0.02  # nm^2
pr.f0 = 1.0e12
pr.fwhm = 100  # nm
pr.order = 1
pr.step = 0.5  # nm

pr.solve_steady_state()
R = pr.R
r = pr.r
fig, ax = plt.subplots()
plt.plot(r, R)
plt.title('Normed R(r)')
params = pr.__dict__
[params.pop(key) for key in ['_R', 'r', 'f', 'n']]
fname = 'example_curve2.txt'
with open(fname, mode='w') as f:
    f.write('r(radius)\tR(growth rate)\n')
    for i in range(R.size):
        f.write(f'{r[i]}\t{R[i]}\n')
    f.write('\nParameters used:\n')
    for key, val in params.items():
        f.write(f'{key}:{val}\n')

Changing parameters and trying to find them via the optimizer

In [ ]:
pr = Experiment2D()
pr.n0 = 2.7  # 1/nm^2
pr.F = 730.0  # 1/nm^2/s
pr.s = 1.0
pr.V = 0.05  # nm^3
pr.tau = 800e-6 # s
pr.D = 3e6 # nm^2/s
pr.sigma = 0.05  # nm^2
pr.f0 = 5.0e8
pr.fwhm = 100  # nm
pr.order = 1
pr.step = 0.5  # nm
optimiser = PSO_Optimizer(pr, 3, 3, 10)
optimiser.set_reference(r, R)
sigma = [0.009, 0.07, 0.002, 'sigma']
D = [1e5, 2e7, 2e4, 'D']
tau = [20e-6, 1000e-6, 20e-6, 'tau']
optimiser.set_variables(sigma, D, tau)
vals_opt = optimiser.optimize_pso()
pr.sigma, pr.D, pr.tau = vals_opt
pr.solve_steady_state()
fig, ax = plt.subplots()
scat = ax.scatter(r, R, label='Reference')
scat1 = ax.scatter(pr.r, pr.R, label='Solution')
plt.show()
print(vals_opt)